# Métodos de Fine-Tuning

### Optimización de Tukey's Fence:
#### 1. Ajustar el valor de k:
##### Reducir 𝑘 puede capturar más outliers, mientras que aumentarlo puede permitir la conservación de datos cercanos a los bordes.
##### Usa visualización de la distribución (boxplots) para entender si los valores extremos realmente deben ser tratados como outliers.

#### 2. Winsorización combinada:
##### En lugar de eliminar los outliers, podrías limitar (winsorize) los valores atípicos a un valor máximo y mínimo razonable.

#### 3. Visualización y análisis contextual:
##### Asegúrate de que los outliers identificados no contengan información importante. Algunos valores atípicos pueden tener significado contextual y no deben eliminarse sin más.

In [6]:
import numpy as np
import pandas as pd

df = pd.read_csv("C:/Users/leons/Downloads/prestamos/rejected_2007_to_2018Q4.csv")

def tukeys_fence(df, column, k=1.5):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - k * IQR
    upper_bound = Q3 + k * IQR
    
    # Filtrar los valores fuera de los límites
    df_filtrado = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    return df_filtrado

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

# Escalar los datos
scaler = StandardScaler()
data_scaled = scaler.fit_transform(df)

# Definir el modelo DBSCAN
dbscan = DBSCAN(eps=0.5, min_samples=5)

# Ajustar el modelo a los datos
dbscan.fit(data_scaled)

# Identificar los outliers (etiquetados como -1 por DBSCAN)
outliers = df[dbscan.labels_ == -1]

## Tengo que soliconar un problema con los floats, pero funciona bien si no los toma en cuenta

In [20]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from collections import Counter

X = df.drop('Zip Code', axis = 1)
y = df['Zip Code']

# Si hay columnas categóricas en X, debemos convertirlas a numéricas
for column in X.columns:
    if X[column].dtype == 'object':  # Si la columna es de tipo object (cadenas de texto)
        le = LabelEncoder()
        X[column] = le.fit_transform(X[column])  # Convertir a valores numéricos

# Convertir la columna objetivo (Zip Code) a numérica si es de tipo cadena
if y.dtype == 'object':
    le_y = LabelEncoder()
    y = le_y.fit_transform(y)  # Convertir Zip Code a valores numéricos

X = X.fillna(X.mean())

print(f"Distribución de clases antes de SMOTE: {Counter(y)}")

smote = SMOTE(k_neighbors = 4)

X_resampled, y_resampled = smote.fit_resample(X, y)

print(X_resampled)
print(y_resampled)

Distribución de clases antes de SMOTE: Counter({np.int64(113): 267102, np.int64(771): 263682, np.int64(301): 260429, np.int64(332): 245496, np.int64(607): 235017, np.int64(751): 221895, np.int64(331): 220306, np.int64(901): 212639, np.int64(105): 211708, np.int64(946): 199190, np.int64(892): 184790, np.int64(70): 173834, np.int64(783): 173029, np.int64(918): 169135, np.int64(926): 155302, np.int64(118): 148735, np.int64(335): 147507, np.int64(605): 143288, np.int64(302): 142456, np.int64(101): 141274, np.int64(192): 139928, np.int64(303): 136841, np.int64(482): 135367, np.int64(853): 132236, np.int64(208): 130709, np.int64(854): 127592, np.int64(213): 127353, np.int64(761): 126134, np.int64(80): 125503, np.int64(774): 124497, np.int64(914): 124160, np.int64(924): 123716, np.int64(329): 122845, np.int64(922): 122665, np.int64(851): 120631, np.int64(442): 116693, np.int64(968): 114466, np.int64(776): 113653, np.int64(602): 112888, np.int64(115): 111780, np.int64(775): 111693, np.int64(21

ValueError: Expected n_neighbors <= n_samples_fit, but n_neighbors = 5, n_samples_fit = 1, n_samples = 1